# Acey Deucey Probability Breakdown

## Creating the Probability Distribution

Let P(A) = Probability that a given bet wins <br>
Need to create a PMF since outcomes are discrete... <br>


$ P(A) = \frac{i-j-1}{13} $ where i,j are high and low card showing, respectively

This formula assumes replacement... which is not realistic, but a good starting point

### Considering Expected Value

Upon first glance, it seems that we should bet when $ P(A) > 50 $%... However, the catch to the game is that you can "post," and lose double your bet. So, instead of betting the probability, we must bet the expected value.

We will now define the return function... <br>
$ R(win) = x $ if i < k < j <br>
$ R(loss) = -x $ if k > i OR k < j <br>
$ R(post) = -2x $ if i < k < j <br>
Where x represents the value of the bet and k represents the value of the third card

### Creating a Counting Strategy to Create Better Distribution

Let's play a test round... <br>
1. Dealer deals a 3 and a King <br>
$ E(A) = \sum_{n=1}^{3}P(n)*R(n) = P(win)*R(win)+P(loss)*R(loss)+P(post)*P(post) $ <br>
$ E(A) = \frac{9}{13} * x + \frac{2}{13} * -x + \frac{2}{13}* -2x = \frac{5x}{13}$ <br>
We have a positive expected return, so we would choose to bet in this scenerion and expect a return of $ \frac{5}{13} $ of our bet. <br>
A. We bet 10, Dealer deals a 5... we win! Return is 10.
2. Dealer deals a 4 and a Queen <br>
$ E(A) = \sum_{n=1}^{3}P(n)*R(n) = P(win)*R(win)+P(loss)*R(loss)+P(post)*P(post) $ <br>
$ E(A) = \frac{7*4-1}{52-5} * x + \frac{4*4-2}{52-5} * -x + \frac{2*4}{52-5}* -2x = \frac{-3x}{47}$ <br>
Negative expected return... we don't bet. Let's look at what's happening here...

$ P(win) = \frac{Cards between i and j}{Cards Remaining} $
$ P(loss) = \frac{Cards outside i and j}{Cards Remaining} $
$ P(post) = \frac{Cards =i and j}{Cards Remaining} $

### Basic Strategy

$ P(win | spread) = \frac{spread-1}{13}$

$0.5 = \frac{spread-1}{13}$

$ 6.5 = spread - 1 $

Spread = 7.5 for P(win) = 0.5... therefore spread must be 8 to be greater that 50% odds without considering posting or non replacement in the deck.

Example 8 card spreads: Queen/4, King/5, High Ace/6, Jack/3, 10/2, 9/Low Ace

So basic strategy will be to bet 8 card spreads with a counting strategy...

### Counting Strategy

Let's use our middle-most betting hand (Queen/4) to define our first negative cards... <br>
Therefore, 5-Jack are all negative value cards. <br>
7,8 have NO chance of hurting us with our betting strategy, so let's define them as -2, and all others as -1.

Now let's look at our positive cards... <br>
Ace will be defined as +2, with 2-4 and Queen, King being +1

|Card|Value|
|---|---|
|Ace|+2|
|2|+1|
|3|+1|
|4|+1|
|5|-1|
|6|-1|
|7|-2|
|8|-2|
|9|-1|
|10|-1|
|Jack|-1|
|Queen|+1|
|King|+1|

# Simulation

In [370]:
from random import randrange

class Card:
    def __init__(self, value, color):
        self.value = value
        self.color = color

In [371]:
print(deck[0].value)

Ace


In [671]:
num_players = 10
min_bet = num_players
count = 0
correct_bets = []

In [709]:
def play_round(num_rounds, spread_to_bet, count_values):
    returns = 0
    correct_bets_round_num = 0
    correct_bets_round_denom = 0
    colors = ['Hearts', 'Diamonds', 'Spades', 'Clubs']
    values = ['Ace', 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
    deck = [Card(value, color) for value in values for color in colors]
    count = 0
    pot = 10
    for r in range(num_rounds):
        print(f'______________________ROUND {r+1}______________________')
        last_cards = []
        for player in range(num_players):
            if len(deck) < 3:
                deck = [Card(value, color) for value in values for color in colors]
                print(f'New Deck!')
                count = 0
            left_card = deck.pop(randrange(len(deck)))
            right_card = deck.pop(randrange(len(deck)))
            print(f'Player {player} is dealt: {left_card.value} of {left_card.color} , {right_card.value} of {right_card.color}')
            count += count_values[left_card.value]
            count += count_values[right_card.value]
            if left_card.value == 'Ace':
                left_card.value = 1
            if right_card.value == 'Ace':
                right_card.value = 14
            spread = abs(right_card.value - left_card.value)
            bet = 0
            outer_cards = [min(right_card.value,left_card.value), max(right_card.value,left_card.value)]
            if player == 1 and not bet_placed:
                if left_card.value <= last_cards[0] or left_card.value >= last_cards[1]:
                    correct_bets_round_num += 1
                    print('Good bet')
                correct_bets_round_denom += 1
            if player == 0:
                if spread >= spread_to_bet:
                    bet_placed = True
                    if count > 0:
                        if spread == 8:
                            ideal_bet = 20
                            bet = min(ideal_bet, pot)
                            deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns)
                        elif spread == 9:
                            ideal_bet = 30
                            bet = min(ideal_bet, pot)
                            deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns)
                        elif spread == 10:
                            ideal_bet = 40
                            bet = min(ideal_bet, pot)
                            deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns)
                        elif spread == 11:
                            ideal_bet = 40
                            bet = min(ideal_bet, pot)
                            deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns)
                        elif spread == 12:
                            ideal_bet = 50
                            bet = min(ideal_bet, pot)
                            deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns)
                        elif spread == 13:
                            bet = pot
                            deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns)
                else:
                    last_cards = outer_cards
                    bet_placed = False
            else:
                if spread == 6:
                    bet = 10
                    deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns)
                elif spread == 7:
                    ideal_bet = 20
                    bet = min(ideal_bet, pot)
                    deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns)
                elif spread == 8:
                    ideal_bet = 30
                    bet = min(ideal_bet, pot)
                    deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns)
                elif spread == 9:
                    ideal_bet = 30
                    bet = min(ideal_bet, pot)
                    deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns)
                elif spread == 10:
                    ideal_bet = 40
                    bet = min(ideal_bet, pot)
                    deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns)
                elif spread == 11:
                    ideal_bet = 50
                    bet = min(ideal_bet, pot)
                    deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns)
                elif spread == 12:
                    ideal_bet = 50
                    bet = min(ideal_bet, pot)
                    deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns)
                elif spread == 13:
                    bet = pot
                    deck, count, pot, returns = deal_middle_card(deck, outer_cards, count, pot, bet, player, returns)
        print(f'Count: {count}')
        returns -= 1
    if correct_bets_round_denom != 0:
        correct_bets.append(correct_bets_round_num / correct_bets_round_denom)
    return returns


def deal_middle_card(deck, outer_cards, count, pot, bet, player, returns):
    middle_card = deck.pop(randrange(len(deck)))
    print(f'Middle Card: {middle_card.value} of {middle_card.color}')
    count += count_values[middle_card.value]
    if middle_card.value == 'Ace':
        if left_card.value == 1:
            middle_card.value = 1
        elif right_card.value == 14:
            middle_card.value = 14
        else:
            middle_card.value = 14
    if outer_cards[0] < middle_card.value and middle_card.value < outer_cards[1]:
        if player == 0:
            returns += bet
        pot -= bet
        if pot < 10:
            pot = 10
        print(f'Bet Won! Player {player} wins ${bet}!')
    elif middle_card.value == outer_cards[0] or middle_card.value == outer_cards[1]:
        if player == 0:
            returns -= bet * 2
        pot += bet * 2
        print(f'POST!!! Suck it player {player}')
    else:
        if player == 0:
            returns -= bet
        pot += bet
    if player == 0:
        print(f'Total Returns: {returns}')
    print(f'Pot Value: ${pot}')
        
    return deck, count, pot, returns

In [710]:
returns = []
spread_to_bet = 8
# count_values = {'Ace': 2, 2:1, 3:1, 4:1, 5:-1, 6:-1, 7:-2, 8:-2, 9:-1, 10:-1, 11:-1, 12:1, 13:1} # Not Great
# count_values = {'Ace': 2, 2:1, 3:1, 4:1, 5:-1, 6:-1, 7:-1, 8:-1, 9:-1, 10:-1, 11:-1, 12:1, 13:1} # 7
count_values = {'Ace': 2, 2:1, 3:1, 4:1, 5:-1, 6:-1, 7:-1, 8:-1, 9:-1, 10:1, 11:1, 12:1, 13:1} # 8 BEST
# count_values = {'Ace': 2, 2:1, 3:1, 4:1, 5:1, 6:-1, 7:-1, 8:-1, 9:-1, 10:-1, 11:-1, 12:1, 13:1} # 6

for i in range(10):
    print(f'------------------------------------------------------TRIAL {i +1}------------------------------------------------------')
    returns.append(play_round(30, spread_to_bet, count_values))

    

------------------------------------------------------TRIAL 1------------------------------------------------------
______________________ROUND 1______________________
Player 0 is dealt: 3 of Clubs , 13 of Clubs
Middle Card: 3 of Hearts
POST!!! Suck it player 0
Total Returns: -20
Pot Value: $30
Player 1 is dealt: 13 of Hearts , 7 of Hearts
Middle Card: 11 of Hearts
Bet Won! Player 1 wins $10!
Pot Value: $20
Player 2 is dealt: 11 of Clubs , 9 of Clubs
Player 3 is dealt: 7 of Diamonds , Ace of Spades
Middle Card: 3 of Spades
Pot Value: $40
Player 4 is dealt: 12 of Hearts , 7 of Spades
Player 5 is dealt: 7 of Clubs , 2 of Diamonds
Player 6 is dealt: 4 of Clubs , 11 of Spades
Middle Card: 10 of Hearts
Bet Won! Player 6 wins $20!
Pot Value: $20
Player 7 is dealt: 2 of Spades , Ace of Diamonds
Middle Card: 12 of Spades
Bet Won! Player 7 wins $20!
Pot Value: $10
Player 8 is dealt: 10 of Diamonds , 9 of Diamonds
Player 9 is dealt: 8 of Hearts , 6 of Hearts
Count: 11
______________________ROUND

Player 6 is dealt: Ace of Diamonds , 10 of Hearts
Middle Card: 8 of Hearts
Bet Won! Player 6 wins $10!
Pot Value: $10
Player 7 is dealt: 13 of Spades , 5 of Diamonds
Middle Card: 2 of Diamonds
Pot Value: $20
Player 8 is dealt: 2 of Spades , 12 of Clubs
Middle Card: 5 of Spades
Bet Won! Player 8 wins $20!
Pot Value: $10
Player 9 is dealt: 11 of Hearts , 7 of Clubs
Count: 6
______________________ROUND 11______________________
Player 0 is dealt: 10 of Diamonds , 13 of Clubs
Player 1 is dealt: 13 of Hearts , 9 of Clubs
Good bet
Player 2 is dealt: 7 of Diamonds , 12 of Hearts
Player 3 is dealt: 3 of Hearts , 13 of Diamonds
Middle Card: 4 of Diamonds
Bet Won! Player 3 wins $10!
Pot Value: $10
Player 4 is dealt: 4 of Spades , 6 of Clubs
Player 5 is dealt: 7 of Spades , Ace of Hearts
Middle Card: 11 of Spades
Bet Won! Player 5 wins $10!
Pot Value: $10
Player 6 is dealt: 7 of Hearts , 12 of Diamonds
Player 7 is dealt: 6 of Hearts , 4 of Clubs
Player 8 is dealt: 6 of Diamonds , 11 of Diamonds
Pl

Pot Value: $10
Player 4 is dealt: 8 of Diamonds , 2 of Diamonds
Middle Card: 11 of Spades
Pot Value: $20
Player 5 is dealt: 12 of Spades , 7 of Spades
Player 6 is dealt: 10 of Spades , 13 of Spades
Player 7 is dealt: 7 of Hearts , 7 of Clubs
Player 8 is dealt: 11 of Clubs , 12 of Diamonds
Player 9 is dealt: 5 of Clubs , 10 of Clubs
Count: 4
______________________ROUND 4______________________
Player 0 is dealt: Ace of Diamonds , 3 of Hearts
Player 1 is dealt: 13 of Diamonds , 6 of Hearts
Good bet
Middle Card: 2 of Spades
Pot Value: $40
Player 2 is dealt: 13 of Clubs , 9 of Hearts
Player 3 is dealt: 13 of Hearts , 6 of Diamonds
Middle Card: 8 of Spades
Bet Won! Player 3 wins $20!
Pot Value: $20
Player 4 is dealt: 12 of Clubs , 10 of Diamonds
Player 5 is dealt: 9 of Clubs , 8 of Clubs
Player 6 is dealt: Ace of Spades , 10 of Hearts
Middle Card: 11 of Diamonds
Pot Value: $40
Player 7 is dealt: 4 of Diamonds , 7 of Diamonds
Player 8 is dealt: 3 of Clubs , 4 of Spades
Player 9 is dealt: 3 of

In [711]:
print(f'Returns Mean: {sum(returns)/len(returns)}')

Returns Mean: -10.0


In [719]:
%%capture
r = []
for i in range(10):
    for i in range(10):
        returns.append(play_round(30, spread_to_bet, count_values))
    r.append(sum(returns)/len(returns))

In [720]:
print(f'Total Winnings: ${round(sum(r)/len(r))}')

Total Winnings: $1


In [714]:
print(correct_bets)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,